In [5]:
# In this notebook, we will define functions extract multiple feature vectors from an image.
# And then concatenate them into a long feature vector
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import glob

def bin_spatial(img, size = (32, 32)):            
    return cv2.resize(img, size).ravel()

def color_hist(img, nbins = 128, bins_range = (0, 1)):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins = nbins, range = bins_range)
    ghist = np.histogram(img[:,:,1], bins = nbins, range = bins_range)
    bhist = np.histogram(img[:,:,2], bins = nbins, range = bins_range)
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def extract_features(imgs, cspace, spatial_size,
                        hist_bins, hist_range, orient, pix_per_cell,
                    cell_per_block, hog_channel, vis, feature_vec, spatial_feat = True, hist_feat = True,
                    hog_feat = True): # Note hist_range may have to change to (0, 1) for png image.
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
        # Read in each one by one
        # apply color conversion if other than 'RGB'
        # Apply bin_spatial() to get spatial color features
        # Apply color_hist() to get color histogram features
        # Apply get_hog_features() to get hog features
        # Append the new feature vector to the features list
    # imgs is a list of path name for the car or notcar images.
    for fname in imgs:
        image = mpimg.imread(fname)
        if cspace != "RGB":
            if cspace == "HSV":
                feat_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == "LUV":
                feat_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == "HLS":
                feat_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == "YUV":
                feat_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == "YCrCb":
                feat_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            feat_image = np.copy(image)
        if spatial_feat is True:
            spatial_features = bin_spatial(feat_image, size = spatial_size) # No need to convert again
            #print(len(spatial_features))
            #plt.plot(spatial_features, 'r'), plt.show()
        if hist_feat is True:
            color_hist_features = color_hist(feat_image, nbins = hist_bins, bins_range = hist_range)
            #print(len(color_hist_features))
            #plt.plot(color_hist_features, 'g'), plt.show()
        if hog_feat is True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feat_image.shape[2]):
                    hog_features.append(get_hog_features(feat_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis, feature_vec))
                hog_features = np.ravel(hog_features)        
                #print(len(hog_features)),
                #plt.plot(hog_features), plt.show()
            else:
                hog_features = get_hog_features(feat_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis, feature_vec)  
        if(spatial_feat is True and hist_feat is True and hog_feat is True):
            features.append(np.concatenate((spatial_features, color_hist_features, hog_features)))
        elif(spatial_feat is True and hist_feat is True):
            features.append(np.concatenate((spatial_features, color_hist_features)))
        elif(spatial_feat is True and hog_feat is True):
            features.append(np.concatenate((spatial_features, hog_features)))
        elif(hist_feat is True and hog_feat is True):
            features.append(np.concatenate((color_hist_features, hog_features)))
        elif(spatial_feat is True and hog_feat is False and hist_feat is False):
            features.append(spatial_features)
        elif(hist_feat is True and hog_feat is False and spatial_feat is False):
            features.append(color_hist_features)
        elif(hog_feat is True and hist_feat is False and spatial_feat is False):
            features.append(hog_features)
    
    # Return list of feature vectors
    return features

def scale_split_features(car_features, noncar_features):
    # Create an array of size (num_samples, num_features_per_sample)
    # Then define a scaler to to compute mean and standard dev of features/
    X = np.vstack((car_features, noncar_features)).astype(np.float64)
    X_scaler = StandardScaler().fit(X)    
    # Define a labels vector based on features lists
    print(len(car_features))
    y = np.hstack((np.ones(len(car_features)), 
              np.zeros(len(notcar_features))))        
    return X_scaler, X, y

    

    
    
    
    